

### Bottleneck層の復元と簡素化 ガタツキの修正 check list     
+ [x]  plan 1 濃くなったがなめらかになったとは言い難い

+ [x]  plan 2 元の状態へのロールバック、もとのほうがよい  

+ [x]  plan 3 good!

+ [x]  plan 4 汚い

+ [x]  plan B 0.5も0.2もあまり変化はない 

Dilated Conv Block (DCB) をシンプルな ResBlock や Conv Block に置き換えた結果、画像全体が崩れてしまったとのこと、承知いたしました。これは、DCBが提供していた**広域の特徴量抽出能力**を失ったことで、モデルが入力画像全体の構造を理解できなくなり、学習が不安定になったためと考えられます。

DCBを完全に削除するのではなく、**その利点を生かしつつガタつきを抑制**するため、以下の2段階のアプローチを推奨します。

-----

## 1\. ⚙️ Bottleneck層の復元と簡素化 (推奨)

最も重要な**Bottleneck層**のDCBは、画像の全体像を捉える上で不可欠です。これを削除すると、デコーダーは局所的なノイズしか拾えなくなります。

Bottleneck層のDCBは維持しつつ、**エンコーダー/デコーダーのDCB**をシンプルな構造に戻すことで、局所的なガタつきを抑制します。

### ✅ 修正案：Bottleneck層は維持し、他のDCBを標準Convに置換

あなたの現在の`UNetGenerator`構造に基づき、以下の修正を提案します。

1.  **Bottleneck層 (`self.bottleneck`):** `dilation=4`のDCBはそのまま維持し、広域の特徴を捉える能力を保ちます。
2.  **Enc3層 (`self.enc3`):** `dilation=2`のDCBを、**通常の畳み込み + 正規化 + ReLU**のシーケンスに置き換えます。これにより、エンコーダーの後半でのガタつきを抑えます。
3.  **Dec3, Dec2層 (`self.dec3`, `self.dec2`):** 同様に、`dilation=2`のDCBを通常の畳み込みに置き換えます。

<!-- end list -->

```python
# UNetGenerator クラス内

# 修正 1: enc3 の DilatedConvBlock(dilation=2) を通常の Conv に置換
self.enc3 = nn.Sequential(
    # nn.Conv2d(128, 256, 3, padding=1) を使用
    nn.Conv2d(128, 256, 3, padding=1),
    nn.InstanceNorm2d(256),
    nn.ReLU(True),
    ResBlock(256)
)
self.pool3 = nn.MaxPool2d(2)


# 修正 2: bottleneck 層は Dilated Conv Block を維持
self.bottleneck = nn.Sequential(
    DilatedConvBlock(256, 512, dilation=4), # 維持
    ResBlock(512),
    DilatedConvBlock(512, 512, dilation=4), # 維持
)


# 修正 3: dec3 の DilatedConvBlock(dilation=2) を通常の Conv に置換
self.dec3 = nn.Sequential(
    # DilatedConvBlock(512, 256, dilation=2) を通常の Conv に置換
    nn.Conv2d(512, 256, 3, padding=1),
    nn.InstanceNorm2d(256),
    nn.ReLU(True),
    ResBlock(256)
)


# 修正 4: dec2 の DilatedConvBlock(dilation=2) を通常の Conv に置換
self.dec2 = nn.Sequential(
    # DilatedConvBlock(256, 128, dilation=2) を通常の Conv に置換
    nn.Conv2d(256, 128, 3, padding=1),
    nn.InstanceNorm2d(128),
    nn.ReLU(True),
    ResBlock(128)
)


# dec1 の DilatedConvBlock(dilation=1) は元々通常の Conv とほぼ同じなので、維持または ResBlock に置換
self.dec1 = nn.Sequential(
    # DilatedConvBlock(128, 64, dilation=1), # 維持または ResBlock に置換
    ResBlock(64) # こちらに置換するとより平滑化に寄与します
)
```

この修正で再度学習を行い、全体構造を保ちながらガタつきが抑制されるか確認してください。

-----

## 2\. 📉 損失関数でガタつきを抑制する

モデル構造の変更と並行して、損失関数の設定を再確認してください。

### B. エッジ損失の重みを下げる

ガタつきはエッジの過剰検出によって起こります。現在 **`1.0 * loss_edge`** としている重みを下げて、モデルがエッジを強く検出することを抑制します。

  * **修正案:** **`0.5`** や **`0.2`** 程度に下げて、メイン損失(`loss_main`)の支配力を強めます。

<!-- end list -->

```python
# トレーニングループ内
loss = loss_main + 0.5 * loss_edge # 1.0 から 0.5 に下げる
```

この調整により、モデルの出力が滑らかになり、ガタつきが目立たなくなる可能性があります。

In [63]:
# train_unet_res_dilated.py
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.optim as optim

In [64]:
# -----------------------
# Residual Block
# -----------------------
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv1 = nn.Conv2d(ch, ch, 3, padding=1)
        self.in1 = nn.InstanceNorm2d(ch)
        self.conv2 = nn.Conv2d(ch, ch, 3, padding=1)
        self.in2 = nn.InstanceNorm2d(ch)

    def forward(self, x):
        h = F.relu(self.in1(self.conv1(x)))
        h = self.in2(self.conv2(h))
        return x + h


# -----------------------
# Dilated Conv Block
# -----------------------
class DilatedConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, dilation=2):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, dilation=dilation, padding=dilation)
        self.inorm = nn.InstanceNorm2d(out_ch)

    def forward(self, x):
        return F.relu(self.inorm(self.conv(x)))


In [73]:
import torch
import torch.nn as nn

# ResBlockとDilatedConvBlockの定義が必要です。ここでは省略します。
# -----------------------
# U-Net Generator (1ch Input / 1ch Output)
# -----------------------
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()

        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1), 
            nn.InstanceNorm2d(64),
            nn.ReLU(True),
            ResBlock(64)
        )
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(True),
            ResBlock(128)
        )
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(
            DilatedConvBlock(128, 256, dilation=2),
            ResBlock(256)
        )
        self.pool3 = nn.MaxPool2d(2)


        self.bottleneck = nn.Sequential(
        DilatedConvBlock(256, 512, dilation=4), # 維持
        ResBlock(512),
        DilatedConvBlock(512, 512, dilation=4), # 維持
        )

        self.up3 = nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1)
        self.dec3 = nn.Sequential(
        # DilatedConvBlock(512, 256, dilation=2) を通常の Conv に置換
        nn.Conv2d(512, 256, 3, padding=1),
        nn.InstanceNorm2d(256),
        nn.ReLU(True),
        ResBlock(256)
        )

        self.up2 = nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1)
        self.dec2 = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(True),
            ResBlock(128)
        )
        
        self.up1 = nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1)
        #self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2, output_padding=1)
        self.dec1 = nn.Sequential(
            DilatedConvBlock(128, 64, dilation=1),
            ResBlock(64)
        )

        self.out_conv = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool1(e1)

        e2 = self.enc2(p1)
        p2 = self.pool2(e2)

        e3 = self.enc3(p2)
        p3 = self.pool3(e3)

        b = self.bottleneck(p3)

        u3 = self.up3(b)
        d3 = self.dec3(torch.cat([u3, e3], dim=1))

        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e2], dim=1))

        u1 = self.up1(d2)
        d1 = self.dec1(torch.cat([u1, e1], dim=1))

        return self.out_conv(d1)

In [74]:
def sobel_edges(x):
    sobel_x = torch.tensor([[-1,0,1],[-2,0,2],[-1,0,1]],
                           dtype=torch.float32, device=x.device).view(1,1,3,3)
    sobel_y = torch.tensor([[-1,-2,-1],[0,0,0],[1,2,1]],
                           dtype=torch.float32, device=x.device).view(1,1,3,3)

    g_x = F.conv2d(x, sobel_x, padding=1)
    g_y = F.conv2d(x, sobel_y, padding=1)

    return torch.sqrt(g_x**2 + g_y**2 + 1e-6)
    
def edge_loss(pred, target):
    return F.l1_loss(sobel_edges(pred), sobel_edges(target))

In [75]:
# Dataset
class SketchDataset(torch.utils.data.Dataset):
    def __init__(self, rough_dir, line_dir, transform=None):
        self.rough_files = sorted(os.listdir(rough_dir))
        self.line_files = sorted(os.listdir(line_dir))
        self.rough_dir = rough_dir
        self.line_dir = line_dir
        self.transform = transform

    def __len__(self):
        return len(self.rough_files)

    def __getitem__(self, idx):
        rough = Image.open(os.path.join(self.rough_dir, self.rough_files[idx])).convert("L")
        line  = Image.open(os.path.join(self.line_dir,  self.line_files[idx])).convert("L")

        if self.transform:
            rough = self.transform(rough)
            line  = self.transform(line)

        return rough, line

In [80]:
# ---------------------------------------------------------------------------
# I. モデルと補助関数の定義 (変更なし)
# ---------------------------------------------------------------------------
# ... UNetGenerator, ResBlock, DilatedConvBlock, sobel_edges, edge_loss の定義はそのまま使用 ...

import torch.nn.functional as F # F.relu, F.conv2d, F.l1_loss のために必要
# ... (他の import はそのまま) ...


# ---------------------------------------------------------------------------
# II. データローディングと設定
# ---------------------------------------------------------------------------
# Dataset クラスはそのまま使用 (paired_transform 引数は不要)

# Transform の修正: Resize を 256x256 に変更
transform = transforms.Compose([
    transforms.Resize((256, 256)), # ★ 128x128 から 256x256 に変更 ★
    transforms.ToTensor(),
])

dataset = SketchDataset("dataset/train/rough", "dataset/train/line", transform)

# DataLoader: 解像度アップに伴い、VRAM節約のためバッチサイズを 2 に下げることを推奨
loader = DataLoader(dataset, batch_size=2, shuffle=True) 

device = "cuda" if torch.cuda.is_available() else "cpu"

model = UNetGenerator(in_channels=1, out_channels=1).to(device)

# ★ 安定性とシャープネスのための損失関数設定 ★
# 損失関数: BCEWithLogitsLoss (安定)
# ★★★ 修正箇所: pos_weight を torch.Tensor として定義 ★★★

# 1. pos_weight の値を Tensor に変換し、デバイスへ送る
# ぼやけ解消のため、7.0〜10.0程度の高い値を使用します
pos_weight_value = 5.0 
pos_weight_tensor = torch.tensor(pos_weight_value, dtype=torch.float).to(device)

# 2. criterion に Tensor 型の pos_weight を渡す
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# オプティマイザ: 学習率を 0.0001 に下げる (暴走防止)
optimizer = optim.Adam(model.parameters(), lr=0.0001) 

os.makedirs("checkpoints", exist_ok=True)


# ---------------------------------------------------------------------------
# III. トレーニングループの修正
# ---------------------------------------------------------------------------
for epoch in range(50):
    total_loss = 0.0
    num_batches = 0
    model.train() 
    
    for rough, line in loader:
        rough, line = rough.to(device), line.to(device)

        line = 1.0 - line
        
        optimizer.zero_grad()
        pred = model(rough)

        # メイン損失 (Logits, Target)
        loss_main = criterion(pred, line) 
        # エッジ損失 (Sigmoid(Logits), Target)
        loss_edge = edge_loss(torch.sigmoid(pred), line) 

        # ★ エッジ重みをシャープネスのため 1.0 に上げる ★
        loss = loss_main + 0.2 * loss_edge

        loss.backward()
        optimizer.step()
        
        # ログ改善
        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}: avg_loss={avg_loss:.4f}")

    torch.save(model.state_dict(), f"checkpoints/unet_1ch_epoch{epoch+1}.pth")

Epoch 1: avg_loss=0.6920
Epoch 2: avg_loss=0.6134
Epoch 3: avg_loss=0.5976
Epoch 4: avg_loss=0.5859
Epoch 5: avg_loss=0.5804
Epoch 6: avg_loss=0.5752
Epoch 7: avg_loss=0.5718
Epoch 8: avg_loss=0.5682
Epoch 9: avg_loss=0.5663
Epoch 10: avg_loss=0.5645
Epoch 11: avg_loss=0.5607
Epoch 12: avg_loss=0.5599
Epoch 13: avg_loss=0.5583
Epoch 14: avg_loss=0.5554
Epoch 15: avg_loss=0.5532
Epoch 16: avg_loss=0.5489
Epoch 17: avg_loss=0.5472
Epoch 18: avg_loss=0.5465
Epoch 19: avg_loss=0.5445
Epoch 20: avg_loss=0.5383
Epoch 21: avg_loss=0.5367
Epoch 22: avg_loss=0.5281
Epoch 23: avg_loss=0.5265
Epoch 24: avg_loss=0.5227
Epoch 25: avg_loss=0.5163
Epoch 26: avg_loss=0.5144
Epoch 27: avg_loss=0.5079
Epoch 28: avg_loss=0.5025
Epoch 29: avg_loss=0.4947
Epoch 30: avg_loss=0.4944
Epoch 31: avg_loss=0.4837
Epoch 32: avg_loss=0.4803
Epoch 33: avg_loss=0.4725
Epoch 34: avg_loss=0.4642
Epoch 35: avg_loss=0.4591
Epoch 36: avg_loss=0.4541
Epoch 37: avg_loss=0.4480
Epoch 38: avg_loss=0.4421
Epoch 39: avg_loss=0.

In [81]:
import torch
import torchvision.transforms.functional as TF
from PIL import Image
# ... (UNetGeneratorの定義が必要) ...

device = "cuda" if torch.cuda.is_available() else "cpu"

model = UNetGenerator(in_channels=1, out_channels=1).to(device)
# チェックポイントのロード...
model.load_state_dict(torch.load("checkpoints/unet_1ch_epoch50.pth", map_location=device))
model.eval()

img = Image.open("test/rough/sample.jpg").convert("L")

# ★★★ リサイズ解像度を 256x256 に変更 ★★★
img = TF.resize(img, (256, 256)) 
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

img = TF.to_tensor(img).unsqueeze(0).to(device)

with torch.no_grad():
    out = torch.sigmoid(model(img))  # 0〜1

out = 1.0 - out 

out = out.clamp(0,1)
out_img = TF.to_pil_image(out[0].cpu())
out_img.save("results/plan3.png")

print("done")

done


result
![result](./results/plan1.png)1
![result](./results/plan3.png)3
![result](./results/plan4.png)4


sample  
![sample](./test/rough/sample.jpg)